In [1]:
import requests
import pandas as pd
import numpy as np
import os
import time
import datetime
import json
from bs4 import BeautifulSoup

In [9]:
data_set = pd.read_csv("data/data_set.csv",index_col=0)

In [3]:
data_set

,movieCd,movieNm,star_score,star_user_count,director,openDt,prdtYear,repNationNm,repGenreNm,showTm,...,d6_show,d7_show,d1_seat,d2_seat,d3_seat,d4_seat,d5_seat,d6_seat,d7_seat,audience
0,20030371,태극기 휘날리며,9.62,77,강제규,20040205,2004,한국,전쟁,148,...,43.7,42.6,64.4,77.1,93.6,95.6,66.3,61.5,61.2,11746135
1,20040555,트로이,9.19,21,볼프강 페터젠,20040521,2004,미국,사극,136,...,30.0,24.7,59.4,80.2,80.5,42.2,50.6,82.3,33.0,3851000
2,20040673,내 머리 속의 지우개,8.00,36,이재한,20041105,2004,한국,멜로/로맨스,117,...,22.7,22.6,38.6,60.4,57.3,25.0,25.1,22.5,27.6,2565078
3,20040649,귀신이 산다,8.80,10,김상진,20040917,2004,한국,코미디,123,...,20.3,19.0,33.7,61.4,61.0,20.2,19.7,18.5,17.7,2890000
4,20159748,투모로우랜드,8.98,136,브래드 버드,20150527,2014,미국,액션,130,...,11.4,10.9,12.8,6.3,8.4,21.9,20.0,4.8,4.0,265744
5,20148807,엣지 오브 투모로우,9.22,366,더그 라이만,20140604,2014,미국,액션,113,...,26.8,27.0,50.4,31.8,54.9,50.1,43.8,13.9,12.6,4699208
6,20040628,우리 형,8.31,14,안권태,20041008,2004,한국,드라마,112,...,26.6,25.8,40.3,59.0,57.2,23.2,24.9,23.9,21.4,2479585
7,20040598,해리포터와 아즈카반의 죄수,8.83,48,알폰소 쿠아론,20040716,2004,미국,드라마,136,...,23.5,20.5,48.5,85.8,73.0,27.1,25.7,24.9,24.0,2532000
8,20040566,슈렉2,7.35,17,켈리 애스버리,20040618,2004,미국,애니메이션,92,...,25.2,23.4,32.9,55.5,58.6,23.3,21.7,20.6,20.6,3300533
9,20030410,실미도,7.68,29,강우석,20031224,2003,한국,전쟁,135,...,27.3,24.5,78.1,80.6,65.2,71.7,76.1,59.4,62.5,11081000


## 데이터 전처리

In [4]:
data_set["openDt"] = data_set["openDt"].apply(lambda x: str(x)[4:6])

In [5]:
def month_change(i):
    if i == "01":
        return 'jan'
    elif i == '02':
        return 'feb'
    elif i == '03':
        return 'mar'
    elif i == '04':
        return 'apr'
    elif i == '05':
        return 'may'
    elif i == '06':
        return 'jun'
    elif i == '07':
        return 'jul'
    elif i == '08':
        return 'aug'
    elif i == '09':
        return 'sep'
    elif i == '10':
        return 'oct'
    elif i == '11':
        return 'nov'
    else:
        return 'dec'
data_set["openDt"] = data_set["openDt"].apply(month_change)    

In [6]:
def grade_change(i):
    if i == '연소자관람가' or i == "모든 관람객이 관람할 수 있는 등급" or i == '전체관람가':
        return 'G'
    elif i == '12세관람가' or i == '12세이상관람가' or i == '국민학생관람불가' or i == '연소자관람불가' or i == '중학생이상관람가' or i == 'nan':
        return 'PG_13'
    elif i == '15세관람가' or i == '15세이상관람가' or i == '15세 미만인 자는 관람할 수 없는 등급 ' or i == '고등학생이상관람가':
        return 'R'
    elif i == '18세관람가' or i == '청소년관람불가':
        return 'NC_17'
    else:
        return i
data_set["watchGradeNm"] = data_set["watchGradeNm"].apply(grade_change)

In [7]:
data_set["showTm"] = data_set["showTm"].astype(int)

In [8]:
def time_change(i):
    if i < 90:
        return 'under_90'
    elif i >= 90 and i < 120:
        return '90_120'
    elif i >=120 and i < 150:
        return '120_150'
    else:
        return '150_up'

data_set["showTm"] = data_set["showTm"].apply(time_change)

In [9]:
def nation_change(i):
    if i == '중국' or i == '홍콩' or i == '대만':
        return 'china'
    elif i == '아이슬란드' or i == '우크라이나' or i == '체코' or i == '노르웨이' or i == ' 오스트리아' or i == '덴마크' or i == '러시아' or i == '이탈리아' or i == '벨기에' or i == '네덜란드' or i == '스페인' or i == '핀란드' or i == '스웨덴' or i == '스위스' or i == '영국' or i == '프랑스' or i == '헝가리' or i == '독일' or i == '아일랜드':
        return 'europe'
    elif i == '태국' or i == '싱가포르' or i == '호주' or i == '캐나다' or i == '페루' or i == '멕시코' or i == '이스라엘' or i == '뉴질랜드' or i == '아르헨티나' or i == '이란' or i == '남아프리카공화국' or i == '인도네시아' or i == '인도':
        return 'other_nation'
    elif i == '한국':
        return 'korea'
    elif i == '미국':
        return 'america'
    elif i == '일본':
        return 'japan'
data_set["repNationNm"] = data_set["repNationNm"].apply(nation_change)

In [10]:
def genre_change(i):
    if i == '드라마' or i == '멜로/로맨스':
        return 'drama_romance'
    elif i == '전쟁' or i == '액션':
        return 'war_action'
    elif i == '공포(호러)' or i == '미스터리':
        return 'horror_mystery'
    elif i == '범죄' or i == '스릴러':
        return 'crime_thriller'
    elif i == 'SF' or i == '판타지' or i == '어드벤처':
        return 'SF_fantasy_adventure'
    elif i == '애니메이션' or i == '가족':
        return 'family_animation'
    elif i == '코미디':
        return 'comedy'
    elif i == '다큐멘터리':
        return 'documentary'
    elif i == '공연' or i == '뮤지컬':
        return 'performance_musical'
    elif i == '사극' or i == '서부극(웨스턴)':
        return 'historical'
    else:
        return i
    
data_set["repGenreNm"] = data_set["repGenreNm"].apply(genre_change)

In [11]:
def company_change(i):
    if i == '소니픽쳐스릴리징월트디즈니스튜디오스코리아(주)' or i == '월트디즈니' or i == '월트디즈니컴퍼니코리아(주)' or i == '월트디즈니코리아㈜':
        return 'walt_disney'
    elif i == '이십세기폭스코리아(주)':
        return 'twentieth_century_fox'
    elif i == '씨제이이앤엠(주)':
        return 'cjenm'
    elif i == '(주)쇼박스':
        return 'showbox'
    elif i == '워너 브러더스 픽쳐스' or i == '워너브러더스 코리아(주)':
        return 'warnerbros'
    elif i == '㈜메가박스' or i == '메가박스(주)플러스엠':
        return 'megabox'
    elif i == '유니버셜 픽쳐스' or i == '유니버설픽쳐스인터내셔널 코리아(유)':
        return 'universal'
    elif i == '(주)넥스트엔터테인먼트월드(NEW)':
        return 'next'
    elif i == '(주)와우픽쳐스':
        return 'wowpictures'
    elif i == '롯데쇼핑㈜롯데엔터테인먼트':
        return 'lotte'
    else:
        return 'other_company'
data_set["companyNm"] = data_set["companyNm"].apply(company_change)

In [12]:
cate_set = data_set[['movieCd', 'director', 'openDt', 'prdtYear', 'repNationNm', 'repGenreNm', 'showTm', 'watchGradeNm', 'actor_1', 'actor_2', 'actor_3', 'companyNm','audience']]

In [13]:
def score_change(i):
    if i < 1000000:
        return 1
    elif i < 2000000 and i >= 1000000:
        return 2
    elif i < 3000000 and i >= 2000000:
        return 3
    elif i < 4000000 and i >= 3000000:
        return 4
    elif i < 5000000 and i >= 4000000:
        return 5
    elif i < 6000000 and i >= 5000000:
        return 6
    elif i < 7000000 and i >= 6000000:
        return 7
    elif i < 8000000 and i >= 7000000:
        return 8
    elif i < 9000000 and i >= 8000000:
        return 9
    else:
        return 10
    
    
cate_set["audience"] = cate_set["audience"].apply(score_change)

/Users/Cho/.pyenv/versions/3.5.1/envs/dss/lib/python3.5/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
def score_change(i):
    if i < 20000:
        return 1
    elif i < 40000 and i >= 20000:
        return 2
    elif i < 60000 and i >= 40000:
        return 3
    elif i < 100000 and i >= 60000:
        return 4
    elif i < 150000 and i >= 100000:
        return 5
    elif i < 250000 and i >= 150000:
        return 6
    elif i < 500000 and i >= 250000:
        return 7
    elif i < 1000000 and i >= 500000:
        return 8
    elif i < 2000000 and i >= 1000000:
        return 9
    else:
        return 10
    
    
cate_set["audience"] = cate_set["audience"].apply(score_change)

/Users/Cho/.pyenv/versions/3.5.1/envs/dss/lib/python3.5/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
cate_set

,movieCd,director,openDt,prdtYear,repNationNm,repGenreNm,showTm,watchGradeNm,actor_1,actor_2,actor_3,companyNm,audience
0,20030371,강제규,feb,2004,korea,war_action,120_150,R,장동건,원빈,이은주,other_company,1
1,20040555,볼프강 페터젠,may,2004,america,historical,120_150,R,다이앤 크루거,올랜도 블룸,에릭 바나,other_company,1
2,20040673,이재한,nov,2004,korea,drama_romance,90_120,PG_13,정우성,손예진,데이비드 맥기니스,other_company,1
3,20040649,김상진,sep,2004,korea,comedy,120_150,PG_13,김응수,장서희,차승원,other_company,1
4,20159748,브래드 버드,may,2014,america,war_action,120_150,PG_13,조지 클루니,휴 로리,브릿 로버트슨,other_company,1
5,20148807,더그 라이만,jun,2014,america,war_action,90_120,PG_13,톰 크루즈,에밀리 블런트,빌 팩스톤,warnerbros,1
6,20040628,안권태,oct,2004,korea,drama_romance,90_120,R,원빈,조성제,신하균,other_company,1
7,20040598,알폰소 쿠아론,jul,2004,america,drama_romance,120_150,G,다니엘 래드클리프,엠마 왓슨,루퍼트 그린트,other_company,1
8,20040566,켈리 애스버리,jun,2004,america,family_animation,90_120,G,마이크 마이어스,카메론 디아즈,에디 머피,other_company,1
9,20030410,강우석,dec,2003,korea,war_action,120_150,R,설경구,안성기,허준호,other_company,1


In [17]:
d1 = cate_set[["director","audience"]]
o1 = cate_set[["openDt","audience"]]
p1 = cate_set[["prdtYear","audience"]]
n1 = cate_set[["repNationNm","audience"]]
g1 = cate_set[["repGenreNm","audience"]]
s1 = cate_set[["showTm","audience"]]
w1 = cate_set[["watchGradeNm","audience"]]
c1 = cate_set[["companyNm","audience"]]

In [18]:
d2 = d1.groupby("director").agg({"audience": np.mean})
o2 = o1.groupby("openDt").agg({"audience": np.mean})
p2 = p1.groupby("prdtYear").agg({"audience": np.mean})
n2 = n1.groupby("repNationNm").agg({"audience": np.mean})
g2 = g1.groupby("repGenreNm").agg({"audience": np.mean})
s2 = s1.groupby("showTm").agg({"audience": np.mean})
w2 = w1.groupby("watchGradeNm").agg({"audience": np.mean})
c2 = c1.groupby("companyNm").agg({"audience": np.mean})

In [19]:
d2 = d2.reset_index()
o2 = o2.reset_index()
p2 = p2.reset_index()
n2 = n2.reset_index()
g2 = g2.reset_index()
s2 = s2.reset_index()
w2 = w2.reset_index()
c2 = c2.reset_index()

In [20]:
d3 = cate_set.merge(d2,how='inner',on='director')
o3 = cate_set.merge(o2,how='inner',on='openDt')
p3 = cate_set.merge(p2,how='inner',on='prdtYear')
n3 = cate_set.merge(n2,how='inner',on='repNationNm')
g3 = cate_set.merge(g2,how='inner',on='repGenreNm')
s3 = cate_set.merge(s2,how='inner',on='showTm')
w3 = cate_set.merge(w2,how='inner',on='watchGradeNm')
c3 = cate_set.merge(c2,how='inner',on='companyNm')

In [21]:
d3 = d3.rename(columns={"audience_y":"director_score"})
o3 = o3.rename(columns={"audience_y":"openDt_score"})
p3 = p3.rename(columns={"audience_y":"prdtYear_score"})
n3 = n3.rename(columns={"audience_y":"repNationNm_score"})
g3 = g3.rename(columns={"audience_y":"repGenreNm_score"})
s3 = s3.rename(columns={"audience_y":"showTm_score"})
w3 = w3.rename(columns={"audience_y":"watchGradeNm_score"})
c3 = c3.rename(columns={"audience_y":"companyNm_score"})

In [22]:
d3 = d3[["movieCd","director_score"]]
o3 = o3[["movieCd","openDt_score"]]
p3 = p3[["movieCd","prdtYear_score"]]
n3 = n3[["movieCd","repNationNm_score"]]
g3 = g3[["movieCd","repGenreNm_score"]]
s3 = s3[["movieCd","showTm_score"]]
w3 = w3[["movieCd","watchGradeNm_score"]]
c3 = c3[["movieCd","companyNm_score"]]

In [23]:
a11 = cate_set[["movieCd","actor_1","audience"]]
a12 = cate_set[["movieCd","actor_2","audience"]]
a13 = cate_set[["movieCd","actor_3","audience"]]

In [24]:
a21 = a11.groupby("actor_1").agg({"audience": np.mean})
a22 = a12.groupby("actor_2").agg({"audience": np.mean})
a23 = a13.groupby("actor_3").agg({"audience": np.mean})

In [25]:
a21 = a21.reset_index()
a22 = a22.reset_index()
a23 = a23.reset_index()

In [26]:
a31 = cate_set.merge(a21,how='outer',on='actor_1')
a32 = cate_set.merge(a22,how='outer',on='actor_2')
a33 = cate_set.merge(a23,how='outer',on='actor_3')

In [27]:
a31 = a31.rename(columns = {"actor_1":"actor"})
a32 = a32.rename(columns = {"actor_2":"actor"})
a33 = a33.rename(columns = {"actor_3":"actor"})

In [28]:
a41 = a31[["actor","audience_y"]]
a42 = a32[["actor","audience_y"]]
a43 = a33[["actor","audience_y"]]

In [29]:
actor = pd.concat([a41,a42,a43])

In [30]:
actor = actor.groupby("actor").agg({"audience_y":np.mean})

In [31]:
actor = actor.reset_index()

In [32]:
a31 = a31[["movieCd","audience_y"]]
a32 = a32[["movieCd","audience_y"]]
a33 = a33[["movieCd","audience_y"]]

In [33]:
actor = a31.merge(a32,how="outer",on="movieCd")

In [34]:
actor = actor.merge(a33,how="outer",on="movieCd")

In [35]:
actor["actor_score"] = actor["audience_y"]+ actor["audience_y_x"] + actor["audience_y_y"]

In [36]:
actor = actor[["movieCd","actor_score"]]

In [37]:
actor = actor.fillna(0)

In [38]:
value_data_set = data_set.drop(['director', 'openDt', 'prdtYear', 'repNationNm', 'repGenreNm', 'showTm', 'watchGradeNm', 'actor_1', 'actor_2', 'actor_3', 'companyNm','audience'],axis=1)

In [40]:
dd_set = value_data_set.merge(d3,how="outer",on="movieCd")

In [41]:
dd_set = dd_set.merge(o3,how="outer",on="movieCd")

In [42]:
dd_set = dd_set.merge(p3,how="outer",on="movieCd")

In [43]:
dd_set = dd_set.merge(n3,how="outer",on="movieCd")

In [44]:
dd_set = dd_set.merge(g3,how="outer",on="movieCd")

In [45]:
dd_set = dd_set.merge(s3,how="outer",on="movieCd")

In [46]:
dd_set = dd_set.merge(w3,how="outer",on="movieCd")

In [47]:
dd_set = dd_set.merge(c3,how="outer",on="movieCd")

In [48]:
dd_set = dd_set.merge(actor,how="outer",on="movieCd")

In [49]:
dd_set["director_score"] = round(dd_set["director_score"],1)
dd_set["openDt_score"] = round(dd_set["openDt_score"],1)
dd_set["prdtYear_score"] = round(dd_set["prdtYear_score"],1)
dd_set["repNationNm_score"] = round(dd_set["repNationNm_score"],1)
dd_set["repGenreNm_score"] = round(dd_set["repGenreNm_score"],1)
dd_set["showTm_score"] = round(dd_set["showTm_score"],1)
dd_set["watchGradeNm_score"] = round(dd_set["watchGradeNm_score"],1)
dd_set["companyNm_score"] = round(dd_set["companyNm_score"],1)
dd_set["actor_score"] = round(dd_set["actor_score"],1)

In [50]:
dd_set = dd_set.drop(["movieCd","movieNm"],axis=1)

In [51]:
dd_set = dd_set.fillna(0)

In [52]:
dd_set

,star_score,star_user_count,preview_audience,d1_audience,d2_audience,d3_audience,d4_audience,d5_audience,d6_audience,d7_audience,...,d7_seat,director_score,openDt_score,prdtYear_score,repNationNm_score,repGenreNm_score,showTm_score,watchGradeNm_score,companyNm_score,actor_score
0,9.62,77,44097,63289,80911,109972,109834,70696,68761,68527,...,61.2,1,1,1,1.0,1,1,1.0,1,3.0
1,9.19,21,59425,78490,120387,111317,47738,64278,120620,39688,...,33.0,1,1,1,1.0,1,1,1.0,1,3.0
2,8.00,36,46,111038,183138,168971,66193,66593,59085,72937,...,27.6,1,1,1,1.0,1,1,1.0,1,3.0
3,8.80,10,57675,77921,153045,145349,45838,44281,41398,38328,...,17.7,1,1,1,1.0,1,1,1.0,1,3.0
4,8.98,136,607,22850,22217,29816,74551,64048,14135,10958,...,4.0,1,1,1,1.0,1,1,1.0,1,3.0
5,9.22,366,80680,377160,246697,469539,433758,363899,105528,95624,...,12.6,1,1,1,1.0,1,1,1.0,1,3.0
6,8.31,14,1254,116069,189865,180242,66331,71244,68921,59488,...,21.4,1,1,1,1.0,1,1,1.0,1,3.0
7,8.83,48,107760,132219,257050,207623,67924,63630,59956,49448,...,24.0,1,1,1,1.0,1,1,1.0,1,3.0
8,7.35,17,49822,86789,157714,159360,56645,51566,49162,43876,...,20.6,1,1,1,1.0,1,1,1.0,1,3.0
9,7.68,29,12718,21361,22498,17411,19854,20871,14694,13525,...,62.5,1,1,1,1.0,1,1,1.0,1,3.0


## dd_set to csv

In [106]:
dd_set.to_csv("value_data.csv",encoding="utf-8")